# **ETL: Limpieza y preparación de los datos**

## **Líbrerias y módulos necesarios**

In [80]:
import warnings
import numpy as np
import pandas as pd
import plotly.express as px
warnings.filterwarnings('ignore')
from funciones import grafico_datos_faltantes

## **Conjunto de datos 1: Cáncer de mama**

```{note}
La información utilizada para este estudio está basada en el Certificado de Defunción del Ministerio de Salud de Colombia 
{cite}`minsalud_defuncion` . En este caso, observaciones sobre casos de defunción relacionados con el cáncer de mama. 

```

Inicialmente, se importa el primer conjunto de datos que contiene información detallada sobre la defunción de pacientes debido al cáncer de mama.

In [81]:
url = "https://raw.githubusercontent.com/sePerezAlbor/Data/refs/heads/main/MAMA8923.csv"
data = pd.read_csv(url, delimiter=",", na_values=[" "], low_memory=False)

Veamos las primeras cinco observaciones para realizar un análisis inicial del conjunto de datos.

In [82]:
data.head()

,COD_DPTO,COD_MUNIC,A_DEFUN,ANO,MES,SEXO,GRU_ED1,EST_CIVIL,CODPTORE,CODMUNRE,...,C_ANT22,C_ANT32,C_PAT2,IDPROFCER,CAUSA_667,P_PMAN_IRIS,CAUSA_MULT,TIPOFORMULARIO,Year,filter_$
0,50,1.0,1,1985,1,2,16,2,50.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1985,Selected
1,15,861.0,2,1985,1,2,24,1,15.0,861.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1985,Selected
2,50,573.0,1,1985,1,2,17,2,50.0,573.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1985,Selected
3,41,1.0,1,1985,1,2,14,2,41.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1985,Selected
4,41,1.0,1,1985,1,2,22,2,41.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1985,Selected


Las primeras cinco observaciones muestran datos con valores faltantes en varias columnas, diferencias en la codificación de variables y posibles inconsistencias en los identificadores de municipios e instituciones de salud. 

In [83]:
data.shape

(78481, 61)

El conjunto de datos inicial cuenta con **61** columnas (variables) y **78481** filas (observaciones).

In [84]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 78481 entries, 0 to 78480
Data columns (total 61 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   COD_DPTO        78481 non-null  int64  
 1   COD_MUNIC       78481 non-null  float64
 2   A_DEFUN         78481 non-null  int64  
 3   ANO             78481 non-null  int64  
 4   MES             78481 non-null  int64  
 5   SEXO            78481 non-null  int64  
 6   GRU_ED1         78481 non-null  int64  
 7   EST_CIVIL       78481 non-null  int64  
 8   CODPTORE        78439 non-null  float64
 9   CODMUNRE        78439 non-null  float64
 10  SIT_DEFUN       78481 non-null  int64  
 11  C_BAS1          78481 non-null  object 
 12  CONS_EXP        78481 non-null  int64  
 13  CAU_HOMOL       78481 non-null  int64  
 14  NOMBRE_ARCHIVO  78481 non-null  object 
 15  GRU_ED2         72563 non-null  float64
 16  AREA_RES        72477 non-null  object 
 17  perman_mun      5705 non-null  

A primera vista, observamos que las variables tienen nombres poco descriptivos, lo que dificulta la comprensión de su significado. Además, muchas de ellas están representadas como valores numéricos de tipo *float64*, aunque en realidad corresponden a categorías. Por ello, procederemos a redefinir los tipos de datos en función del significado de cada variable. También identificamos una alta presencia de datos faltantes en algunas columnas (incluyendo dos columnas que en su totalidad son datos faltantes), lo que requerirá un tratamiento adecuado.

In [85]:
column_types = {
    "COD_DPTO": "object", "COD_MUNIC": "object", "A_DEFUN": "object", "ANO": "object", "MES": "object", "SEXO": "object",
    "GRU_ED1": "object", "EST_CIVIL": "object", "CODPTORE": "object", "CODMUNRE": "object", "SIT_DEFUN": "object", "C_BAS1": "object",
    "CONS_EXP": "object", "CAU_HOMOL": "object", "AREA_RES": "object", "PMAN_MUER": "object", "COD_INST": "object", "NOM_INST": "object",
    "NIVEL_EDU": "object", "CODPRES": "object", "SEG_SOCIAL": "object", "MAN_MUER": "object", "CODOCUR": "object", "CODMUNOC": "object",
    "C_MUERTE": "object", "ASIS_MED": "object", "C_DIR1": "object", "C_ANT1": "object", "C_ANT2": "object", "C_ANT3": "object",
    "C_PAT1": "object", "C_MCM1": "object", "CAUSA_666": "object", "SIMUERTEPO": "object", "OCUPACION": "object", "IDPERTET": "object",
    "IDADMISALUD": "object", "IDCLASADMI": "object", "C_DIR12": "object", "C_ANT12": "object", "C_ANT22": "object", "C_ANT32": "object",
    "C_PAT2": "object", "IDPROFCER": "object", "Year": "object", "filter_$": "object"
}


In [86]:
data = pd.read_csv(url, delimiter=",", na_values=[" "], dtype = column_types)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 78481 entries, 0 to 78480
Data columns (total 61 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   COD_DPTO        78481 non-null  object 
 1   COD_MUNIC       78481 non-null  object 
 2   A_DEFUN         78481 non-null  object 
 3   ANO             78481 non-null  object 
 4   MES             78481 non-null  object 
 5   SEXO            78481 non-null  object 
 6   GRU_ED1         78481 non-null  object 
 7   EST_CIVIL       78481 non-null  object 
 8   CODPTORE        78439 non-null  object 
 9   CODMUNRE        78439 non-null  object 
 10  SIT_DEFUN       78481 non-null  object 
 11  C_BAS1          78481 non-null  object 
 12  CONS_EXP        78481 non-null  object 
 13  CAU_HOMOL       78481 non-null  object 
 14  NOMBRE_ARCHIVO  78481 non-null  object 
 15  GRU_ED2         72563 non-null  float64
 16  AREA_RES        72477 non-null  object 
 17  perman_mun      5705 non-null  

Nótese que el cambio se ha realizado exitosamente.

In [87]:
data = data.drop(columns=[
    "NOMBRE_ARCHIVO",
    "GRU_ED2",
    "perman_mun",
    "tiem_per",
    "EST_CIVM",
    "OTRSITIODE",
    "HORA",
    "C_MUERTEB",
    "C_MUERTEC",
    "C_MUERTED",
    "C_MUERTEE",
    "CAUSA_667",
    "P_PMAN_IRIS",
    "CAUSA_MULT",
    "TIPOFORMULARIO"
])


### **Datos Faltantes**

Observemos a detalle los datos faltantes en este conjunto de datos.

:::{note}
En esta sección del proyecto (**ETL**), nos enfocaremos únicamente en la identificación y estandarización de los valores faltantes dentro del conjunto de datos. Para ello, se detectarán las variables con datos ausentes y se reemplazarán valores inconsistentes, como **"Sin información"**, **9** etc, por **NaN**, asegurando una representación uniforme de los datos faltantes. Además, se evaluará la cantidad de valores nulos en cada columna para determinar si es necesario eliminar aquellas con un porcentaje significativo de datos ausentes. Estas transformaciones permitirán mejorar la calidad del dataset y facilitar su posterior análisis en la fase de **EDA**.
:::

In [88]:
missing_percent = (data.isnull().sum() / len(data)) * 100
missing_percent = missing_percent.sort_values(ascending=False)
missing_percent[missing_percent > 0]

CODMUNOC       95.974822
CODOCUR        95.974822
C_MCM1         95.871612
C_ANT32        95.572177
C_DIR12        95.293128
SIMUERTEPO     94.782177
C_ANT22        94.770709
C_ANT12        94.426677
C_PAT2         94.196047
MAN_MUER       91.963660
C_ANT3         84.475223
C_PAT1         83.720901
C_MUERTE       73.861189
IDCLASADMI     67.640575
COD_INST       63.969623
NOM_INST       63.765752
C_ANT2         62.891655
OCUPACION      52.438170
C_ANT1         47.900766
CAUSA_666      44.622265
C_DIR1         40.389394
IDADMISALUD    39.748474
PMAN_MUER      38.576216
IDPERTET       38.452619
IDPROFCER      38.452619
CODPRES        22.374842
ASIS_MED       16.488067
SEG_SOCIAL     16.488067
NIVEL_EDU      16.488067
AREA_RES        7.650259
CODPTORE        0.053516
CODMUNRE        0.053516
dtype: float64

In [89]:
missing_count = missing_percent[missing_percent > 0].count()
print("La cantidad de variables con datos faltantes es de: ", missing_count)
without_missing = missing_percent[missing_percent == 0].count()
print("La cantidad de variables sin datos faltantes es de: ", without_missing)
missing_above_70 = missing_percent[missing_percent > 70]
print("La cantidad de variables con más del 70% de datos faltantes es de: ", missing_above_70.count())
missing_below_70 = missing_percent[missing_percent <= 70]
print("La cantidad de variables con menos del 70% de datos faltantes es de: ", missing_below_70.count())


La cantidad de variables con datos faltantes es de:  32
La cantidad de variables sin datos faltantes es de:  14
La cantidad de variables con más del 70% de datos faltantes es de:  13
La cantidad de variables con menos del 70% de datos faltantes es de:  33


Veamos esto gráficamente:

In [90]:
grafico_datos_faltantes(data)

Al observar el gráfico, se puede decir que algunas variables presentan un alto porcentaje de datos faltantes, con varias de ellas superando el 90%, como `codcur` y `codmunoc`, que tienen un 95.97% de datos ausentes. Otras variables también muestran valores significativos de ausencia, como `SIMUERTEPO` (94.78%). Sin embargo, a medida que avanzamos en la gráfica, notamos que ciertas variables tienen menor cantidad de datos faltantes, con algunas en torno al 22%. Finalmente, hay un conjunto de variables sin valores faltantes (0%), lo que indica que esos datos están completamente disponibles en el conjunto analizado.

### **Eliminación de variables**


Para optimizar el análisis, eliminamos ciertas variables que no aportan valor significativo. La decisión se tomó con base en los siguientes criterios:  



**1. Variables eliminadas por recomendación de expertos**  
Los profesionales encargados del conjunto de datos recomendaron eliminar estas variables, ya que en este conjunto de datos NO son relevantes para el estudio y están representadas por otras variables:  
(`CAU_HOMOL`, `filter_$`)  



**2. Variables eliminadas por más del 75% de datos faltantes**  
Estas variables tienen un porcentaje muy alto de valores nulos, lo que dificulta su uso sin introducir sesgos en el análisis:  
**Observación:** Debido al alto porcentaje de datos faltantes, estas variables no pueden ser imputadas de manera confiable. Además, su eliminación no afecta el análisis, ya que no aportan información relevante o pueden derivarse de otras variables presentes en el conjunto de datos.

| Variable      | Porcentaje de Datos Faltantes |
|---------------|-------------------------------|
| `CODMUNOC`    | 95.97%                        |
| `CODOCUR`     | 95.97%                        |
| `C_MCM1`      | 95.87%                        |
| `C_ANT32`      | 95.57%                        |
| `C_DIR12`     | 95.29%                        |
| `SIMUERTEPO`  | 94.78%                        |
| `C_ANT22`     | 94.77%                        |
| `C_ANT12`     | 94.43%                        |
| `C_PAT2`      | 94.20%                        |
| `MAN_MUER`    | 91.96%                        |
| `C_ANT3`      | 84.48%                        |
| `C_PAT1`      | 83.72%                        |



**3. Variables que no son relevantes para el estudio**
Estas variables NO serán utilizadas en el enfoque de este estudio y, además, presentan una cantidad significativa de valores nulos:
| Variable     | Motivo de Eliminación                                                                                               | Porcentaje de Valores Nulos |
|--------------|----------------------------------------------------------------------------------------------------------------------|------------------------------|
| `A_DEFUN`    | No aporta información relevante                                                                                      | 0%                           |
| `ANO`        | Ya se cuenta con una variable relacionada al año.                                                                    | 0%                           |
| `AREA_RES`   | Información no necesaria                                                                                              | 7.65%                        |
| `C_ANT2`     | Ya se cuenta con una variable relacionada a los antecendentes. Alto número de valores faltantes                      | 62.89%                       |
| `COD_INST`   | Código de institución poco útil. Alto porcentaje de datos faltantes. Quizás esta información de las instituciones sea antigua debido a las fechas en las que fueron tomadas estas observaciones. | 63.97%                       |
| `CODMUNRE`   | Código no relevante para el análisis                                                                                  | 0.05%                        |
| `CODPRES`    | Código no relevante para el análisis                                                                                  | 22.37%                       |
| `CONS_EXP`   | Código sin valor significativo en el estudio                                                                         | 0%                           |
| `CONS_EXP`   | Certificación médica. Variable sin relevancia en el estudio                                                          | 0%                           |
| `IDCLASADMI` | No aporta valor significativo y alto porcentaje de datos faltantes                                                   | 67.64%                       |
| `IDPERTET`   | Ya se cuenta con una variable relacionada al grupo étnico (CODPTORE). Además, alto número de valores faltantes       | 38.45%                       |
| `IDPROFCER`  | No aporta valor significativo                                                                                        | 38.45%                       |
| `NOM_INST`   | Ocurre del mismo modo que el código de la institución. No se puede imputar porque no se podría garantizar la institución. | 63.77%                       |
| `OCUPACION`  | Ocupación. Alto número de valores faltantes                                                                          | 52.44%                       |




In [92]:
data = data.drop(columns=[
    # 1. Variables eliminadas por recomendación de expertos
    'CAU_HOMOL', 'filter_$',

    # 2. Variables eliminadas por más del 75% de datos faltantes
    'CODMUNOC', 'CODOCUR', 'C_MCM1', 'C_ANT32', 'C_DIR12',
    'SIMUERTEPO', 'C_ANT22', 'C_ANT12', 'C_PAT2', 'MAN_MUER',
    'C_ANT3', 'C_PAT1',

    # 3. Variables no relevantes para el estudio
    'A_DEFUN', 'ANO', 'AREA_RES', 'C_ANT2', 'COD_INST', 'CODMUNRE',
    'CODPRES', 'CONS_EXP', 'IDCLASADMI', 'IDPERTET', 'IDPROFCER',
    'NOM_INST', 'OCUPACION'
])


In [93]:
data.shape

(78481, 19)

Nótese que la eliminación se realizó de forma exitosa y ahora el conjunto de datos cuenta con solo **19** variables.

### **Renombrado de variables**

In [94]:
data = data.rename(columns={
    'COD_DPTO': 'depto_ocurr',       # Departamento donde ocurrió la defunción (DIVIPOLA)
    'COD_MUNIC': 'munic_ocurr',      # Municipio donde ocurrió la defunción (DIVIPOLA)
    'MES': 'mes_def',                # Mes de la defunción
    'SEXO': 'sexo',                  # Sexo del fallecido
    'GRU_ED1': 'grupo_edad',         # Grupo de edad del fallecido
    'EST_CIVIL': 'estado_civil',     # Estado civil del fallecido
    'CODPTORE': 'grupo_etnico',      # Pertenencia étnica según cultura/pueblo/rasgos físicos (DIVIPOLA)
    'SIT_DEFUN': 'sitio_def',        # Sitio donde ocurrió la defunción
    'C_BAS1': 'causa_basica',        # Código de la causa básica de la defunción
    'PMAN_MUER': 'prob_muerte',      # Probable manera de muerte
    'NIVEL_EDU': 'nivel_edu',        # Nivel educativo del fallecido
    'SEG_SOCIAL': 'seg_social',      # Régimen de seguridad social
    'C_MUERTE': 'cert_medica',       # Cómo se determinó la causa de muerte
    'ASIS_MED': 'asistencia_med',    # Recibió asistencia médica antes de fallecer
    'C_DIR1': 'causa_directa',       # Causa directa de la defunción
    'C_ANT1': 'causa_ant_1',         # Causa antecedente 1
    'CAUSA_666': 'causa_ops',        # Causa agrupada según la lista OPS 6/67
    'IDADMISALUD': 'ent_salud_cod',  # Código de la Entidad Administradora en Salud
    'Year': 'año_def'                # Año en que ocurrió la defunción
})


In [95]:
data.head()

,depto_ocurr,munic_ocurr,mes_def,sexo,grupo_edad,estado_civil,grupo_etnico,sitio_def,causa_basica,prob_muerte,nivel_edu,seg_social,cert_medica,asistencia_med,causa_directa,causa_ant_1,causa_ops,ent_salud_cod,año_def
0,50,001,01,2,16,2,50,2,1749,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1985
1,15,861,01,2,24,1,15,2,1749,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1985
2,50,573,01,2,17,2,50,2,1749,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1985
3,41,001,01,2,14,2,41,1,1749,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1985
4,41,001,01,2,22,2,41,2,1749,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1985


Ahora los nombres de cada variable son más legibles y ordenados. Lo que facilita el análisis de estos datos. [Diccionario de Variables](https://kmarcela11.github.io/ProyectoFinal_SeminarioInvestigativo/variables.html)

In [96]:
grafico_datos_faltantes(data)

### **Re-estructuración de categorías por variables**

En esta sección, se llevarán a cabo ajustes para mejorar la claridad y usabilidad de los datos. Primero, se renombrarán las variables con nombres más legibles y comprensibles, facilitando su interpretación. Además, las categorías etiquetadas como **"Sin información"** en algunas variables serán convertidas a **NaN**, lo que permitirá un análisis más estructurado, evitando sesgos y facilitando la imputación de datos en etapas posteriores. Estas transformaciones contribuirán a una mejor comprensión y optimización del conjunto de datos para su análisis.


#### **Variable: sexo**

In [97]:
data['sexo'].value_counts(dropna=False)

sexo
2    78481
Name: count, dtype: int64

In [98]:
data['sexo'] = data['sexo'].astype(str)
data['sexo'].replace('2', 'Femenino', inplace=True)
data['sexo'].value_counts(dropna=False)

sexo
Femenino    78481
Name: count, dtype: int64

#### **Variable: Mes de defunción**

In [99]:
mes_dict_str = {
    "01": "Enero", "02": "Febrero", "03": "Marzo", "04": "Abril",
    "05": "Mayo", "06": "Junio", "07": "Julio", "08": "Agosto",
    "09": "Septiembre", "10": "Octubre", "11": "Noviembre", "12": "Diciembre"
}
data['mes_def'] = data['mes_def'].astype(str).str.zfill(2).replace(mes_dict_str)
data['mes_def'].value_counts(dropna=False)


mes_def
Diciembre     6782
Mayo          6765
Octubre       6724
Julio         6672
Agosto        6642
Junio         6540
Marzo         6493
Enero         6431
Septiembre    6426
Noviembre     6415
Abril         6379
Febrero       6212
Name: count, dtype: int64

#### **Variable: Estado civil**

In [100]:
data['estado_civil'].value_counts(dropna=False)
data['estado_civil'] = data['estado_civil'].astype(str)

In [101]:
data['estado_civil'] = data['estado_civil'].replace(['5', '6', '9'], np.nan)

estado_civil_dict = {
    "1": "Soltero",
    "2": "Casado",
    "3": "Viudo",
    "4": "Unión Libre, Divorciado/Otro"
}

data['estado_civil'] = data['estado_civil'].replace(estado_civil_dict)


In [102]:
data['estado_civil'].value_counts(dropna=False)

estado_civil
NaN                             23529
Casado                          17136
Soltero                         14058
Unión Libre, Divorciado/Otro    12878
Viudo                           10880
Name: count, dtype: int64

In [103]:
proporciones = data['estado_civil'].value_counts(normalize=True)
proporciones = proporciones[~proporciones.index.isna()]  # Excluir NaN si aparece
num_nan = data['estado_civil'].isna().sum()

In [104]:
valores_imputados = np.random.choice(
    proporciones.index,         # categorías (ej. Casado, Soltero, etc.)
    size=num_nan,               # cuántos valores imputar
    p=proporciones.values       # probabilidades según proporción
)

In [105]:
data.loc[data['estado_civil'].isna(), 'estado_civil'] = valores_imputados
data['estado_civil'].value_counts(dropna=False)

estado_civil
Casado                          24435
Soltero                         20087
Unión Libre, Divorciado/Otro    18400
Viudo                           15559
Name: count, dtype: int64

#### **Variable: Sitio de defunción**

In [106]:
data['sitio_def'].value_counts(dropna = False)

sitio_def
1    43365
2    27522
3     6844
4      695
6       41
5       13
9        1
Name: count, dtype: int64

In [107]:
data['sitio_def'] = data['sitio_def'].astype(str)
data['sitio_def'] = data['sitio_def'].replace( ['4','5', '6', '9'], np.nan)

sitio_def_dict = {"1": "Hospital o Clínica", "2": "Casa", "3": "Otro Sitio"}
data['sitio_def'] = data['sitio_def'].replace(sitio_def_dict)

data.loc[data['sitio_def'].isna(), 'sitio_def'] = "Otro Sitio"

In [108]:
data['sitio_def'].value_counts()

sitio_def
Hospital o Clínica    43365
Casa                  27522
Otro Sitio             7594
Name: count, dtype: int64

#### **Variable: Nivel educativo**

In [109]:
data['nivel_edu'] = data['nivel_edu'].replace(
    ['6', '7', '8', '9', '99', '10', '11', '12', '13'], pd.NA)

nivel_edu_dict = {
    "1": "Preescolar", "2": "Primaria", "3": "Secundaria",
    "4": "Superior", "5": "Ninguno"}

data['nivel_edu'] = data['nivel_edu'].replace(nivel_edu_dict)

data['nivel_edu'].fillna("No especificado", inplace=True)



In [110]:
data['nivel_edu'].value_counts(dropna = False)

nivel_edu
No especificado    25840
Primaria           25365
Secundaria         15637
Superior            7268
Ninguno             3838
Preescolar           533
Name: count, dtype: int64

#### **Variable: Causa agrupada con base en la Lista 6/67 de la OP**

In [111]:
data['causa_ops'].value_counts(dropna = False)

causa_ops
208      37004
NaN      35020
208.0     6457
Name: count, dtype: int64

In [112]:
data['causa_ops'] = data['causa_ops'].replace([208, '208.0', '208'], "Tumor Maligno")

data['causa_ops'].fillna("Tumor Maligno", inplace=True)

data['causa_ops'].value_counts(dropna=False)

causa_ops
Tumor Maligno    78481
Name: count, dtype: int64

In [113]:
data.drop('causa_ops', axis=1, inplace=True)

#### **Variable: Asistencia médica**

In [114]:
data['asistencia_med'] = data['asistencia_med'].replace('9', '2')

asistencia_med_dict = {'1': "SI", '2': "NO", '3': "Ignorado"}

data['asistencia_med'] = data['asistencia_med'].map(asistencia_med_dict)

data['asistencia_med'].fillna('NO', inplace=True)
data['asistencia_med'].value_counts(dropna=False)


asistencia_med
SI          58251
NO          20136
Ignorado       94
Name: count, dtype: int64

#### **Variable: Probable manera de muerte**

In [115]:
data['prob_muerte'].value_counts(dropna=False)

prob_muerte
1      48200
NaN    30275
3          6
Name: count, dtype: int64

In [116]:
data.drop('prob_muerte', axis=1, inplace=True)

#### **Variable: Grupo de edad**

In [117]:
data['grupo_edad'] = data['grupo_edad'].astype(str)

data['grupo_edad'] = data['grupo_edad'].replace(['7', '8', '9', '26', '27', '28'], '25')


In [118]:
grupo_edad_dict = {
    "10": "15-19 años", "11": "20-24 años", "12": "25-29 años",
    "13": "30-34 años", "14": "35-39 años", "15": "40-44 años",
    "16": "45-49 años", "17": "50-54 años", "18": "55-59 años",
    "19": "60-64 años", "20": "65-69 años", "21": "70-74 años",
    "22": "75-79 años", "23": "80-84 años", "24": "85+ años",
    "25": "Edad desconocida"
}

data['grupo_edad'] = data['grupo_edad'].astype(str).replace(grupo_edad_dict)


In [119]:
data['grupo_edad'].value_counts(dropna = False)

grupo_edad
55-59 años          9373
60-64 años          9309
50-54 años          8708
65-69 años          8291
70-74 años          7473
45-49 años          6927
75-79 años          6151
80-84 años          4969
40-44 años          4880
85+ años            4554
Edad desconocida    3260
35-39 años          2778
30-34 años          1334
25-29 años           363
20-24 años            91
15-19 años            20
Name: count, dtype: int64

#### **Variable: Seguridad social**

In [120]:
data['seg_social'].replace('9', np.nan, inplace=True)

seg_social_dict = {
    "1": "Contributivo", "2": "Subsidiado", "3": "Vinculado",
    "4": "Particular", "5": "Otro", "6": "Ignorado"
}
data['seg_social'] = data['seg_social'].astype(str).replace(seg_social_dict)

data['seg_social'].replace('nan', np.nan, inplace=True)

data['seg_social'].fillna("Otro", inplace=True)

data['seg_social'].value_counts(dropna = False)

seg_social
Contributivo    33467
Subsidiado      24760
Otro            15287
Vinculado        3916
Particular        927
Ignorado          124
Name: count, dtype: int64

#### **Variable: cert_medica - ¿Cómo se determino la muerte?**

In [121]:
data['cert_medica'].value_counts(dropna = False)

cert_medica
NaN    57967
2      13906
"       3145
4       2439
9        769
1        214
3         41
Name: count, dtype: int64

In [122]:
data.drop('cert_medica', axis=1, inplace=True)

#### **Variable: Código de la entidad administradora en salud a la que pertenecía el fallecido**

In [123]:
data['ent_salud_cod'].value_counts(dropna = False)


ent_salud_cod
NaN    31195
1      24180
2      20012
5       2221
9        515
4        203
3        114
"         41
Name: count, dtype: int64

In [124]:
data['ent_salud_cod'].replace('9', np.nan, inplace = True)

In [125]:
ent_salud_dict = {
    "1": "EPS", "2": "EPS - Subsidiado", "3": "EAPB",
    "4": "ESE", "5": "EESS"}
data['ent_salud_cod'] = data['ent_salud_cod'].astype(str).replace(ent_salud_dict)

In [126]:
data.drop('ent_salud_cod', axis=1, inplace=True)

#### **Variable: Grupo étnico**

In [127]:
data.drop('grupo_etnico', axis=1, inplace=True)

#### **Variable: municipio de defunción**

In [128]:
data["municipio"] = data["depto_ocurr"].astype(str) + data["munic_ocurr"].astype(str).str.zfill(3)

In [129]:
data.shape

(78481, 15)

## **Conjunto de datos 2: DIVIPOLA**


Este conjunto de datos proporciona los códigos oficiales utilizados para identificar departamentos, municipios y otras divisiones territoriales. Estos códigos sirven como referencia en el Conjunto de Datos 1 para estandarizar la información geográfica y facilitar su análisis.

In [130]:
column_types = {
    "Código Departamento": "object",
    "Código Municipio": "object",
    "Código Centro Poblado": "object",
    "Nombre Departamento": "object",
    "Nombre Municipio": "object",
    "Nombre Centro Poblado	": "object",
    "Tipo": "object"
}
url = "https://raw.githubusercontent.com/sePerezAlbor/Data/refs/heads/main/Divipola.csv"
data2 = pd.read_csv(url, delimiter = ",", na_values = [" "], dtype = column_types)
data2.head(15)

,Código Departamento,Código Municipio,Código Centro Poblado,Nombre Departamento,Nombre Municipio,Nombre Centro Poblado,Tipo
0,05,05001,05001000,ANTIOQUIA,MEDELLÍN,MEDELLÍN,CM
1,05,05001,05001001,ANTIOQUIA,MEDELLÍN,PALMITAS,C
2,05,05001,05001004,ANTIOQUIA,MEDELLÍN,SANTA ELENA,C
3,05,05001,05001005,ANTIOQUIA,MEDELLÍN,PEDREGAL ALTO,IPM
4,05,05001,05001009,ANTIOQUIA,MEDELLÍN,ALTAVISTA,C
5,05,05001,05001010,ANTIOQUIA,MEDELLÍN,AGUAS FRÍAS,CP
6,05,05001,05001012,ANTIOQUIA,MEDELLÍN,LA LOMA,CP
7,05,05001,05001013,ANTIOQUIA,MEDELLÍN,SAN JOSÉ DEL MANZANILLO,CP
8,05,05001,05001014,ANTIOQUIA,MEDELLÍN,BARRO BLANCO,CP
9,05,05001,05001015,ANTIOQUIA,MEDELLÍN,EL CERRO,CP


In [ ]:
data2.shape

(9205, 7)

In [ ]:
data2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9205 entries, 0 to 9204
Data columns (total 7 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   Código Departamento    9205 non-null   object
 1   Código Municipio       9205 non-null   object
 2   Código Centro Poblado  9205 non-null   object
 3   Nombre Departamento    9205 non-null   object
 4   Nombre Municipio       9205 non-null   object
 5   Nombre Centro Poblado  9205 non-null   object
 6   Tipo                   9205 non-null   object
dtypes: object(7)
memory usage: 503.5+ KB


Nótese que se cuenta con un conjunto de datos bastante completo. 

In [131]:
condiciones = [
    data['depto_ocurr'].isin(['91', '18', '94', '95', '86', '97']),
    data['depto_ocurr'].isin(['5', '11', '15', '17', '25', '41', '54', '63', '66', '68', '73']),
    data['depto_ocurr'].isin(['8', '13', '20', '23', '44', '47', '70', '88']),
    data['depto_ocurr'].isin(['81', '85', '50', '99']),
    data['depto_ocurr'].isin(['19', '27', '52', '76'])
]

regiones = ['Amazonía', 'Andina', 'Caribe', 'Orinoquía', 'Pacífica']

data['region'] = np.select(condiciones, regiones, default='Otra')


In [132]:
data['region'].value_counts(dropna=False)

region
Andina       48416
Caribe       14026
Pacífica     13947
Orinoquía     1582
Amazonía       510
Name: count, dtype: int64

In [135]:
codigos = data['munic_ocurr']
#Limpieza: quitar decimales, ceros a la izquierda y convertir a strings de 5 dígitos
def normalizar_codigo(c):
    try:
        c = str(int(float(c)))  # convierte float a int (ej: '1.0' -> 1 -> '1')
        return c       # llena con ceros a la izquierda para tener 5 dígitos
    except:
        return None

codigos_limpios = list(set([normalizar_codigo(c) for c in codigos if normalizar_codigo(c)]))

In [138]:
# Separar válidos e inválidos
divipola_oficial = data2['Código Municipio'].astype(str).tolist()
validos = [c for c in codigos_limpios if c in divipola_oficial]
invalidos = [c for c in codigos_limpios if c not in divipola_oficial]

print("Códigos válidos:", validos)
print("Códigos inválidos:", invalidos)


Códigos válidos: []
Códigos inválidos: ['672', '394', '743', '577', '54', '520', '537', '649', '432', '234', '895', '36', '215', '816', '212', '887', '217', '390', '332', '319', '787', '321', '548', '52', '111', '798', '350', '473', '32', '877', '322', '349', '750', '545', '59', '503', '496', '238', '720', '540', '147', '614', '690', '742', '689', '326', '579', '851', '854', '418', '175', '152', '290', '209', '62', '398', '53', '421', '221', '109', '411', '898', '335', '713', '788', '435', '628', '682', '124', '235', '533', '75', '762', '664', '809', '283', '810', '318', '154', '680', '858', '176', '497', '514', '41', '430', '145', '161', '302', '211', '736', '793', '523', '763', '140', '15', '674', '480', '717', '570', '489', '764', '645', '576', '878', '297', '385', '240', '745', '99', '610', '219', '560', '678', '13', '78', '665', '466', '575', '843', '250', '670', '279', '55', '980', '667', '817', '518', '135', '870', '511', '162', '324', '214', '847', '361', '769', '298', '392', '

## **Conjunto de datos final**

Tras analizar los dos conjuntos de datos previamente, procederemos a reemplazar los códigos de departamentos y municipios utilizando el conjunto de datos ``DIVIPOLA``. El objetivo de este proceso es mejorar la claridad y comprensión de la información, asignando a cada código su departamento y municipio correspondiente. Esto permitirá un análisis más preciso, estructurado y fácil de interpretar en el conjunto de datos general.  


In [ ]:
departamento_dict = dict(zip(data2['Código Departamento'], data2['Nombre Departamento']))
municipio_dict = dict(zip(data2['Código Municipio'], data2['Nombre Municipio']))
data['Nombre_Departamento'] = data['depto_ocurr'].map(departamento_dict)
data['Nombre_Municipio'] = data['municipio'].map(municipio_dict)
data = data.drop(columns = ['munic_ocurr', 'municipio', 'depto_ocurr']) 
data.head()


,mes_def,sexo,grupo_edad,estado_civil,sitio_def,causa_basica,nivel_edu,seg_social,asistencia_med,causa_directa,causa_ant_1,año_def,region,Nombre_Departamento,Nombre_Municipio
0,Enero,Femenino,45-49 años,Casado,Casa,1749,No especificado,Otro,NO,NaN,NaN,1985,Orinoquía,META,VILLAVICENCIO
1,Enero,Femenino,85+ años,Soltero,Casa,1749,No especificado,Otro,NO,NaN,NaN,1985,Andina,BOYACÁ,VENTAQUEMADA
2,Enero,Femenino,50-54 años,Casado,Casa,1749,No especificado,Otro,NO,NaN,NaN,1985,Orinoquía,META,PUERTO LÓPEZ
3,Enero,Femenino,35-39 años,Casado,Hospital o Clínica,1749,No especificado,Otro,NO,NaN,NaN,1985,Andina,HUILA,NEIVA
4,Enero,Femenino,75-79 años,Casado,Casa,1749,No especificado,Otro,NO,NaN,NaN,1985,Andina,HUILA,NEIVA


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 78481 entries, 0 to 78480
Data columns (total 15 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   mes_def              78481 non-null  object
 1   sexo                 78481 non-null  object
 2   grupo_edad           78481 non-null  object
 3   estado_civil         78481 non-null  object
 4   sitio_def            78481 non-null  object
 5   causa_basica         78481 non-null  object
 6   nivel_edu            78481 non-null  object
 7   seg_social           78481 non-null  object
 8   asistencia_med       78481 non-null  object
 9   causa_directa        46783 non-null  object
 10  causa_ant_1          40888 non-null  object
 11  año_def              78481 non-null  object
 12  region               78481 non-null  object
 13  Nombre_Departamento  60880 non-null  object
 14  Nombre_Municipio     52033 non-null  object
dtypes: object(15)
memory usage: 9.0+ MB


In [ ]:
columnas_ordenadas = [
    'Nombre_Departamento', 'Nombre_Municipio', 'sitio_def', 'año_def', 
    'mes_def', 'sexo', 'estado_civil', 'grupo_edad', 'nivel_edu', 'seg_social',  'asistencia_med',
    'causa_directa', 'causa_ant_1',  'causa_basica', 'region']
data_new = data[columnas_ordenadas]



In [ ]:
data_new.head()

,Nombre_Departamento,Nombre_Municipio,sitio_def,año_def,mes_def,sexo,estado_civil,grupo_edad,nivel_edu,seg_social,asistencia_med,causa_directa,causa_ant_1,causa_basica,region
0,META,VILLAVICENCIO,Casa,1985,Enero,Femenino,Casado,45-49 años,No especificado,Otro,NO,NaN,NaN,1749,Orinoquía
1,BOYACÁ,VENTAQUEMADA,Casa,1985,Enero,Femenino,Soltero,85+ años,No especificado,Otro,NO,NaN,NaN,1749,Andina
2,META,PUERTO LÓPEZ,Casa,1985,Enero,Femenino,Casado,50-54 años,No especificado,Otro,NO,NaN,NaN,1749,Orinoquía
3,HUILA,NEIVA,Hospital o Clínica,1985,Enero,Femenino,Casado,35-39 años,No especificado,Otro,NO,NaN,NaN,1749,Andina
4,HUILA,NEIVA,Casa,1985,Enero,Femenino,Casado,75-79 años,No especificado,Otro,NO,NaN,NaN,1749,Andina


Finalmente, obtenemos un conjunto de datos más claro, comprensible y estructurado, lo que facilita su interpretación y lo deja listo para el análisis exploratorio.  


In [ ]:
grafico_datos_faltantes(data_new)

In [ ]:
data_new.to_excel('C:\\Users\\kamac\\OneDrive\\Desktop\\SeminarioInvestigativoUN\\docs.xlsx', index = False, engine = 'openpyxl')

```{note}

En el siguiente enlace se encuentra el diccionario de variables utilizado en este proyecto: [Diccionario de Variables](https://kmarcela11.github.io/ProyectoFinal_SeminarioInvestigativo/variables.html)
Este diccionario incluye una descripción detallada de cada una de las variables presentes en el conjunto de datos, facilitando su comprensión y el análisis del proyecto.

```

## **Referencias**
```{bibliography} references.bib
:style: plain
